# AI-Book-Recommender: Model Development
This notebook implements the recommendation models: popularity-based, content-based, collaborative filtering with SVD, and a hybrid system combining all three.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse.linalg import svds

# Load cleaned datasets
books = pd.read_csv('books.csv', low_memory=False)
ratings = pd.read_csv('ratings.csv')

In [ ]:
# Popularity-Based Recommender
def popularity_recommender(ratings, books, top_n=10):
    pop_scores = ratings['ISBN'].value_counts().head(top_n).index
    return books[books['ISBN'].isin(pop_scores)][['ISBN', 'Book-Title', 'Book-Author', 'Publisher']]

In [ ]:
# Content-Based Recommender
def content_based_recommender(book_title, books, top_n=10):
    tfidf = TfidfVectorizer(stop_words='english')
    tfidf_matrix = tfidf.fit_transform(books['Book-Title'])
    cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
    indices = pd.Series(books.index, index=books['Book-Title']).drop_duplicates()
    idx = indices[book_title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[1:top_n+1]
    book_indices = [i[0] for i in sim_scores]
    return books.iloc[book_indices][['ISBN', 'Book-Title', 'Book-Author', 'Publisher']]

In [ ]:
# Collaborative Filtering with SVD
def svd_recommender(ratings, user_id, books, top_n=10, k=50):
    user_item_matrix = ratings.pivot(index='User-ID', columns='ISBN', values='Book-Rating').fillna(0)
    user_ratings_mean = user_item_matrix.mean(axis=1)
    R_demeaned = user_item_matrix.sub(user_ratings_mean, axis=0)
    U, sigma, Vt = svds(R_demeaned, k=k)
    sigma = np.diag(sigma)
    all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) + user_ratings_mean.values.reshape(-1, 1)
    preds_df = pd.DataFrame(all_user_predicted_ratings, columns=user_item_matrix.columns, index=user_item_matrix.index)
    sorted_user_predictions = preds_df.loc[user_id].sort_values(ascending=False)
    user_data = ratings[ratings['User-ID'] == user_id]['ISBN'].values
    recommendations = sorted_user_predictions[~sorted_user_predictions.index.isin(user_data)].head(top_n)
    return books[books['ISBN'].isin(recommendations.index)][['ISBN', 'Book-Title', 'Book-Author', 'Publisher']]

In [ ]:
# Hybrid Recommender
def hybrid_recommender(user_id, book_title, ratings, books, top_n=10):
    print('Content-Based:')
    print(content_based_recommender(book_title, books, top_n))
    print('\nCollaborative Filtering:')
    print(svd_recommender(ratings, user_id, books, top_n))
    print('\nPopularity-Based:')
    print(popularity_recommender(ratings, books, top_n))